In [70]:
from transformers import AutoConfig, MBartForConditionalGeneration, MBart50TokenizerFast
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint

config = AutoConfig.from_pretrained(
        "facebook/mbart-large-50",
        #decoder_start_token_id = 0,
        early_stopping = False,
        no_repeat_ngram_size = 0,
        #forced_bos_token_id= None,
    )
    
tokenizer_kwargs = {
    "use_fast": True,
    "additional_special_tokens": ['<obj>', '<subj>', '<triplet>'],
    "src_lang": "nl_XX",
    "tgt_lang": "nl_XX"
}

tokenizer = MBart50TokenizerFast.from_pretrained(
    "facebook/mbart-large-50",
    **tokenizer_kwargs
)


model = MBartForConditionalGeneration.from_pretrained(
    "facebook/mbart-large-50",
    config=config
)

model.config.decoder_start_token_id = tokenizer.lang_code_to_id["nl_XX"]

# forced_bos_token_id = (
#         tokenizer.lang_code_to_id["nl_XX"]
#     )
# model.config.forced_bos_token_id = forced_bos_token_id

#model.config.decoder_start_token_id = tokenizer.lang_code_to_id[data_args.lang]

    # if not conf.finetune:
model.resize_token_embeddings(len(tokenizer))


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Embedding(250057, 1024)

In [98]:
dir(model)

['T_destination',
 '__annotations__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_apply',
 '_auto_class',
 '_backward_compatibility_gradient_checkpointing',
 '_backward_hooks',
 '_buffers',
 '_call_impl',
 '_can_retrieve_inputs_from_name',
 '_convert_head_mask_to_5d',
 '_create_or_get_repo',
 '_expand_inputs_for_generation',
 '_forward_hooks',
 '_forward_pre_hooks',
 '_from_config',
 '_get_backward_hooks',
 '_get_decoder_start_token_id',
 '_get_logits_processor',
 '_get_logits_warper',
 '_get_name',
 '_get_repo_url_from_name',
 '_get_resized_embeddings',
 '_get_resized_lm_head',
 '_get_stopping_criteria',
 '_hook_rss_

In [25]:
from datasets import load_dataset, set_caching_enabled

train_file = r'C:\Users\kbrekelm\Desktop\python_projects\crocodile\out\nl\rebel_20000-25000.jsonl'
validation_file = r'C:\Users\kbrekelm\Desktop\python_projects\crocodile\out\nl/rebel_30000-35000.jsonl'
test_file = r'C:\Users\kbrekelm\Desktop\python_projects\crocodile\out\nl\rebel_35000-40000.jsonl'


datasets = load_dataset(r"C:\Users\kbrekelm\Desktop\python_projects\rebel_dutch\datasets\rebel-short.py", data_files={'train': train_file, 'dev': validation_file, 'test': test_file})



Using custom data configuration default-f5a48e90d46f958c


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset rebel-short downloaded and prepared to C:\Users\kbrekelm\.cache\huggingface\datasets\rebel-short\default-f5a48e90d46f958c\0.0.0\59a61d620608097d70b188bb7ff16e936c5b79675e44362a5c1dc1831ca5ca2d. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [35]:
datasets["train"][0]

{'id': 'Q52454-1',
 'title': 'Tennis Hall of Fame',
 'context': 'De International Tennis Hall of Fame , een museum gelegen in de stad Newport in de Amerikaanse staat Rhode Island , werd door James Van Alen in 1954 gesticht op de plaats waar in 1881 de eerste Amerikaanse nationale kampioenschappen werden gehouden . ',
 'triplets': '<triplet> Newport <subj> Amerikaanse <obj> land <triplet> Amerikaanse <subj> Rhode Island <obj> omvat bestuurlijke eenheid <triplet> Rhode Island <subj> Amerikaanse <obj> land <subj> Amerikaanse <obj> gelegen in bestuurlijke eenheid'}

In [71]:
from torch import as_tensor


model_inputs = tokenizer(datasets["train"][0]["context"], max_length=256, padding=True, truncation=True, return_tensors="pt")
labels = tokenizer(datasets["train"][0]["triplets"], max_length=128, padding=True, truncation=True, return_tensors="pt")

In [72]:
generated = model.generate(**model_inputs)

In [97]:
tokenizer.batch_decode(generated)


['nl_XX<s> De International Tennis Hall of Fame, een museum gelegen in de stad Newport in de Amerikaanse staat Rhode Island, werd door James Van Alen in 1954 gesticht op de plaats waar in 1881 de eerste Amerikaanse nationale kampioenschappen werden gehouden...</s>']

In [96]:
tokenizer.batch_decode(model_inputs["input_ids"])
#remove <s> token


['nl_XX De International Tennis Hall of Fame, een museum gelegen in de stad Newport in de Amerikaanse staat Rhode Island, werd door James Van Alen in 1954 gesticht op de plaats waar in 1881 de eerste Amerikaanse nationale kampioenschappen werden gehouden. </s>']

In [95]:
decoded_label = tokenizer.batch_decode(labels["input_ids"])
decoded_label


['nl_XX<triplet> Newport <subj> Amerikaanse <obj> land <triplet> Amerikaanse <subj> Rhode Island <obj> omvat bestuurlijke eenheid <triplet> Rhode Island <subj> Amerikaanse <obj> land <subj> Amerikaanse <obj> gelegen in bestuurlijke eenheid</s>']

In [88]:
def extract_triplets(text):
    triplets = []
    relation, subject, relation, object_ = '', '', '', ''
    text = text.strip()
    current = 'x'
    text = text.replace("nl_XX", "")
    for token in text.replace("<s>", "").replace("<pad>", "").replace("</s>", "").split():
        if token == "<triplet>":
            current = 't'
            if relation != '':
                triplets.append({'head': subject.strip(), 'type': relation.strip(),'tail': object_.strip()})
                relation = ''
            subject = ''
        elif token == "<subj>":
            current = 's'
            if relation != '':
                triplets.append({'head': subject.strip(), 'type': relation.strip(),'tail': object_.strip()})
            object_ = ''
        elif token == "<obj>":
            current = 'o'
            relation = ''
        else:
            if current == 't':
                subject += ' ' + token
            elif current == 's':
                object_ += ' ' + token
            elif current == 'o':
                relation += ' ' + token
    if subject != '' and relation != '' and object_ != '':
        triplets.append({'head': subject.strip(), 'type': relation.strip(),'tail': object_.strip()})
    return triplets


In [89]:
extract_triplets(decoded_label[0])

[{'head': 'Newport', 'type': 'land', 'tail': 'Amerikaanse'},
 {'head': 'Amerikaanse',
  'type': 'omvat bestuurlijke eenheid',
  'tail': 'Rhode Island'},
 {'head': 'Rhode Island', 'type': 'land', 'tail': 'Amerikaanse'},
 {'head': 'Rhode Island',
  'type': 'gelegen in bestuurlijke eenheid',
  'tail': 'Amerikaanse'}]

In [59]:
generated_tokens = model.generate(
    **model_inputs,
    forced_bos_token_id=tokenizer.lang_code_to_id["nl_XX"]
)


AttributeError: 'list' object has no attribute 'shape'

In [24]:
tokenizer.batch_decode(generated_tokens, skip_special_tokens=False)


['<s>nl_XXፕሬፕሬፕሬፕሬፕሬፕሬፕሬፕሬፕሬፕሬፕሬፕሬፕሬፕሬፕሬፕሬፕሬፕሬፕሬፕሬፕሬፕሬፕሬፕሬፕሬፕሬፕሬፕሬፕሬፕሬፕሬፕሬፕሬፕሬፕሬፕሬፕሬፕሬፕሬፕሬፕሬፕሬፕሬፕሬፕሬፕሬፕሬፕሬፕሬፕሬፕሬፕሬፕሬፕሬፕሬፕሬፕሬፕሬፕሬፕሬፕሬፕሬፕሬፕሬፕሬፕሬፕሬፕሬፕሬፕሬፕሬፕሬፕሬፕሬፕሬፕሬፕሬፕሬፕሬፕሬፕሬፕሬፕሬፕሬፕሬፕሬፕሬፕሬ ଲାଭ ଲାଭ ଲାଭ ଲାଭ ଲାଭ ଲାଭ ଲାଭ ଲାଭ ଲାଭ ଲାଭ ଲାଭ ଲାଭ ଲାଭ ଲାଭ ଲାଭ ଲାଭ ଲାଭ ଲାଭ ଲାଭ ଲାଭ ଲାଭ ଲାଭ ଲାଭ ଲାଭ ଲାଭ ଲାଭ ଲାଭ ଲାଭ ଲାଭ ଲାଭ ଲାଭ ଲାଭ ଲାଭ ଲାଭ ଲାଭ ଲାଭ ଲାଭ ଲାଭ ଲାଭ ଲାଭ ଲାଭ ଲାଭ ଲାଭ ଲାଭ ଲାଭ ଲାଭ ଲାଭ ଲାଭ ଲାଭ ଲାଭ ଲାଭ ଲାଭ ଲାଭ ଲାଭ ଲାଭ ଲାଭ ଲାଭ ଲାଭ ଲାଭ ଲାଭ ଲାଭ ଲାଭ ଲାଭ ଲାଭ ଲାଭ ଲାଭ ଲାଭ ଲାଭ ଲାଭ ଲାଭ ଲାଭ ଲାଭ ଲାଭ ଲାଭ ଲାଭ ଲାଭ ଲାଭ ଲାଭ pozostaw pozostaw pozostaw pozostaw pozostaw pozostaw pozostaw pozostaw pozostaw pozostaw pozostaw pozostaw pozostaw pozostaw pozostaw pozostaw pozostaw pozostaw pozostaw pozostaw pozostaw pozostaw pozostaw pozostaw pozostaw pozostaw pozostaw pozostaw pozostaw pozostaw pozostaw</s>']

In [30]:
datasets["train"][0]

{'id': 'Q52454-1',
 'title': 'Tennis Hall of Fame',
 'context': 'De International Tennis Hall of Fame , een museum gelegen in de stad Newport in de Amerikaanse staat Rhode Island , werd door James Van Alen in 1954 gesticht op de plaats waar in 1881 de eerste Amerikaanse nationale kampioenschappen werden gehouden . ',
 'triplets': '<triplet> Newport <subj> Amerikaanse <obj> land <triplet> Amerikaanse <subj> Rhode Island <obj> omvat bestuurlijke eenheid <triplet> Rhode Island <subj> Amerikaanse <obj> land <subj> Amerikaanse <obj> gelegen in bestuurlijke eenheid'}

In [31]:
generated_tokens = model.generate(
    datasets["train"][0],
    forced_bos_token_id=tokenizer.lang_code_to_id["nl_XX"]
)

AttributeError: 'dict' object has no attribute 'shape'